In [1]:
import tdmclient.notebook
await tdmclient.notebook.start()
from tdmclient import ClientAsync, aw
client = ClientAsync()
client.process_waiting_messages()
node = await client.wait_for_node()
aw(node.run())
aw(node.stop())
aw(node.unlock())

import time
import cv2
from l42 import *
from local_nav import *
from VisGraph import *
from update_pos import *
from kalman_filter import *


In [2]:
#getters and setters functions

@tdmclient.notebook.sync_var
def get_prox_value():
    global prox_horizontal
    return prox_horizontal

#@tdmclient.notebook.sync_var
#def get_ground_value():
#    global prox_ground_reflected
#    return prox_ground_reflected

@tdmclient.notebook.sync_var
def get_left_speed():
    global motor_left_target
    return motor_left_target

@tdmclient.notebook.sync_var
def get_right_speed():
    global motor_right_target
    return motor_right_target

@tdmclient.notebook.sync_var
def set_motor_speed(left_speed, right_speed):
    global motor_left_target, motor_right_target
    motor_left_target = left_speed
    motor_right_target = right_speed

In [3]:
VideoCap = cv2.VideoCapture(0)
ret, frame = VideoCap.read()

#Create constant variables
next_goal_trigger = 20
delta_t = 0.05
start = []
stop = []

#Creation of the map and the global path
while(len(stop)==0 or (len(start)==0)):
    ret, frame_init = VideoCap.read()
    start, stop = detect_start_stop(frame_init)
    
global_path = initialisation(frame, start, stop)
start = global_path[0]
stop = global_path[len(global_path)-1]
    
#Set the initial motor speeds at 0
set_motor_speed(0, 0)

#Initialisation of the variables
prox_sensors = np.array([0, 0, 0, 0, 0, 0, 0])

#Initialisation Kalman filter
kalman_filter = KalmanFilter(delta_t)

#Find the initial position and orientation of the robot
position, orientation = detect_thymio(start, 0, 0, 0, kalman_filter, frame)

#Initialisation of the variables that will keep in memory the position and orientation of the last iteration
old_position = position
old_orientation = orientation

#Initialisation of the variables that will keep track of the current goal position and orientation
current_goal = 1
current_goal_pos = global_path[current_goal]

distance_to_goal, current_goal_orientation = calculation_distance_and_angle(position, current_goal_pos)

#Initialisation of the variables that will be updated to set the speed of the wheels
left_speed = 0
right_speed = 0

#Initialisation of the boolean that will recall if the camera detection is possible or not at each iteration
possible_detection = True

while(True):
    ret, frame = VideoCap.read()
    current_goal_pos = global_path[current_goal]
    #Recall the values of the sensors and the wheel speeds 
    prox_sensors = get_prox_value()
    #prox_ground_values = get_ground_value()
    #set_motor_speed(100, 100)
    
    #Reset and update the detection boolean
    #possible_detection = #fonction prévue à cet effet
    
    #if possible_detection :
    #Vision : Returns the position and the orientation of the robot from the camera
        
    position, orientation = detect_thymio(old_position, old_orientation, left_speed, right_speed, kalman_filter, frame)
    cv2.circle(frame, (int(position[0]), int(position[1])), 10, (255, 0, 0), 2)
    x2 = position[0] + 50 * np.cos(orientation)
    y2 = position[1] + 50 * np.sin(orientation)
    cv2.line(frame, (int(position[0]), int(position[1])), (int(x2), int(y2)),(0, 255, 0), 2)
    for i in range (len(global_path)-1):
        cv2.line(frame, (int(global_path[i][0]), int(global_path[i][1])), (int(global_path[i+1][0]), int(global_path[i+1][1])),(0, 0, 255), 2)
    cv2.imshow('image', frame)

    #else :
        #The position and orientation of the robot is estimated from its last state and the wheel speeds
        #position = kalman(position, orientation, left_speed, right_speed) #Ou peu importe la forme de la fonction 
        #orientation = kalman(old_position, orientation, left_speed, right_speed) #Ou peu importe la forme de la fonction
    
    #Update the current goal orientation and the distance from the robot
    distance_to_goal, current_goal_orientation = calculation_distance_and_angle(position, current_goal_pos)

    #Find the speeds to apply to the robot
    left_speed, right_speed = localNavigation(orientation, old_orientation, current_goal_orientation, prox_sensors)
    #Apply the desired speeds to the robot
    set_motor_speed(int(left_speed), int(right_speed))
    
    #Keep in memory the position and orientation of the robot for the next 
    old_position = position
    old_orientation = orientation
    
    #Update the current goal if necessary
    if (distance_to_goal < next_goal_trigger) :
        current_goal += 1
    
    if (current_goal == len(global_path)) :
        set_motor_speed(0, 0)
        break
    if cv2.waitKey(1)&0xFF==ord('q'):
        break
    await client.sleep(0.05)

VideoCap.release()
cv2.destroyAllWindows()



bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué
bloqué


100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 181.56it/s]


In [4]:
await tdmclient.notebook.stop()